In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pprint import pprint

In [11]:
pd.set_option('display.max_columns', None)

In [12]:
data = pd.read_csv("kunstveiling_nl_txmm.csv")

In [13]:
data.describe(include="all")

,web-scraper-order,web-scraper-start-url,pagination,pagination-href,price,kunstenaar,beschrijving,image-src,url,detail_page,detail_page-href,height_painting_cm,width_painting_cm,condition,technique,signed,framed,height_frame_cm,width_frame_cm,period,style,subject,name,price_incl
count,8910,8910,8910,8910,6342.000000,8539,8902,8744,8910,8910,8910,3032.000000,3032.000000,4486,4486,4487,4485,4487.000000,4487.000000,6267,2692,2912,8910,1271
unique,8910,1,744,804,NaN,2678,8061,8742,8910,1,8910,NaN,NaN,5,8,3,4,NaN,NaN,5,13,16,8526,220
top,1607951272-977,https://www.kunstveiling.nl/veilingopbrengsten...,578,https://www.kunstveiling.nl/veilingopbrengsten...,NaN,Martin van Wordragen (1928-2011),LET OP: Dit item wordt geveild door Kunstveili...,"data:image/svg+xml,%3Csvg width='500' height='...","""url"": ""https://www.kunstveiling.nl/items/jas-...",Bekijk item,https://www.kunstveiling.nl/items/mary-waters-...,NaN,NaN,Zeer Goed,Olieverf,Handgesigneerd,Ingelijst,NaN,NaN,1945 t/m 1999,Abstract,Abstract,"Henri Plaat - Gouache, Zonder titel - Ingelijst","172,50"
freq,1,8910,13,12,NaN,235,47,3,1,8910,1,NaN,NaN,2502,2567,4284,2980,NaN,NaN,3424,646,647,14,86
mean,NaN,NaN,NaN,NaN,363.443866,NaN,NaN,NaN,NaN,NaN,NaN,65.338061,68.957454,NaN,NaN,NaN,NaN,78.764208,84.289057,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,636.639148,NaN,NaN,NaN,NaN,NaN,NaN,35.719177,37.454636,NaN,NaN,NaN,NaN,36.257102,44.211076,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,150.000000,NaN,NaN,NaN,NaN,NaN,NaN,40.000000,41.000000,NaN,NaN,NaN,NaN,52.000000,55.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,225.000000,NaN,NaN,NaN,NaN,NaN,NaN,60.000000,60.000000,NaN,NaN,NaN,NaN,73.000000,79.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,375.000000,NaN,NaN,NaN,NaN,NaN,NaN,83.000000,90.000000,NaN,NaN,NaN,NaN,100.000000,104.000000,NaN,NaN,NaN,NaN,NaN


### Explanation
Above are some basic statistics are shown of the web scraped data for the website **kunstveilig.nl**. The lowest number of rows available is for the feature style (2692). To be able to perform an ablation study on the features, we will have to adjust by dropping all rows that contain NaN values. 

The following preprocess steps will need to be executed:
- First, we will try to extract style values from the description. This will be done based on the already present other values for style.
- Secondly, all rows where price is **NaN** will have to be dropped. Without a price, this study is pointless. We are trying to see the effect of different features on prices.

In [14]:
all_styles = [x for x in data['style'].unique() if isinstance(x, str)]
all_styles = list(map(lambda style: style.lower(), all_styles))
print(all_styles)

['expressionistisch', 'klassiek', 'realistisch', 'modern', 'impressionistisch', 'abstract', 'surrealistisch', 'street art', 'minimalistisch', 'afrikaans', 'oriëntaals', 'romantisch', 'aziatisch']


In [15]:
for description in data.loc[data['style'].isnull() & data['beschrijving'].str.lower().str.contains("|".join(all_styles), regex=True)]['beschrijving'].values:
    print(description)

Artist: Simeon Gonzales


Title: Pisco Y Mazanas


Medium: Acryl on canvas


Size: 120 cm x 100 cm ,Gesigneerd + certificaat van echtheid.





Dit schilderij zal opgerold verzonden worden in een stevige koker met verzekering en Track & Trace, tevens is het mogelijk om het schilderij door ons opgespannen te laten worden op een Classic 65 mm spielatten frame, vraag gerust naar de mogelijkheden ook wat verzenden en/of bezorgen aan huis betreft.

Simeón González Ayquipa in 1973 geboren in Cusco, Peru  is uitgegroeid tot een wonderkind van de kunst. Zijn werk tart de grenzen van de cultuur en de beperkingen van tijd.Simeón González werk is een grote bijdrage aan het abstract expressionisme.
Dit werk van Strik komt uit een serie die ontstond n.a.v. een reis die hij maakte door een berggebied in de Spaanse provincie Rioja. En inderdaad, de warme, verzadigde kleuren kunnen, ook voor wie daar niet geweest is, opgevat worden als abstracte manifestaties van een woestijnachtig, door de zon gestoo

In [16]:
for description in data.loc[data['price'].isnull() & data['beschrijving'].str.contains('€|startprijs', regex=True)]['beschrijving'].values:
    print(description)

Origineel werk van kunstenaar Gena. Wordt geleverd met een certificaat van echtheid. Acrylverf op doek.Zijkanten zijn mee geschilderd, zodat geen lijst nodig - direct op te hangen.Het schilderij is voorzien van een vernislaag, dus extra beschermd tegen stof, vuil en vochtinvloed! Formaat : 80 x 160 cm, 3D.Galerie prijs is €1400,-Verzendkosten Nederland, Belgie = €60,-
Origineel werk van kunstenaar Gena. Wordt geleverd met een certificaat van echtheid. Acrylverf op doek.Zijkanten zijn mee geschilderd, zodat geen lijst nodig - direct op te hangen.Het schilderij is voorzien van een vernislaag, dus extra beschermd tegen stof, vuil en vochtinvloed! Formaat : 60 x 140 cm,3DVerzendkosten Nederland, België =€45,-
BLOEMEN IN ROOD

Gouache, dekkend en in een prachtig rood met acrylverf geschilderd op vel papier.


 

* afmetingen: 50 x 50 cm


* in lade bewaard gebleven en in nog prima conditie.


* rechtsonder in potlood gesigneerd: HEK '82




Bieden vanaf € 60  (ex verzendkosten / alleen verz

Having checked for both the style and price in the description, we can conclude that with a rule-based extraction method it is infeasible. Manual labor would be an option, but given the amount of work ahead we choose to leave it for now. Almost all **NaN** values are from old auctions, while kunstveiling.nl's new auctions are well structured and created in vast amounts.

After the full data analysis, kunstveiling.nl will be scraped again in order to acquire as much data as possible.

In [20]:
data = data.drop(axis=1, labels=['price_incl']).dropna()

After we dropped **price_incl** and used dropna, which was not a neccessary column to begin with, we were left with 650 samples that provided fully populated feature vectors.

Now let's export it back to a csv file to be used for the auto-encoder and clustering algorithm.

In [21]:
data

,web-scraper-order,web-scraper-start-url,pagination,pagination-href,price,kunstenaar,beschrijving,image-src,url,detail_page,detail_page-href,height_painting_cm,width_painting_cm,condition,technique,signed,framed,height_frame_cm,width_frame_cm,period,style,subject,name
0,1607959998-4764,https://www.kunstveiling.nl/veilingopbrengsten...,266,https://www.kunstveiling.nl/veilingopbrengsten...,527.0,Matthieu Wiegman (1886-1971),Mattheus Johannes Marie 'Matthieu' WiegmanZwol...,https://images.kunstveiling.nl/tmp-8d702894b02...,"""url"": ""https://www.kunstveiling.nl/items/olie...",Bekijk item,https://www.kunstveiling.nl/items/matthieu-wie...,50.0,40.0,Goed,Olieverf,Handgesigneerd,Ingelijst,60.0,50.0,1900 t/m 1944,Expressionistisch,Stilleven,Matthieu Wiegman - Olieverf - Bergense School ...
2,1607954744-2485,https://www.kunstveiling.nl/veilingopbrengsten...,32,https://www.kunstveiling.nl/veilingopbrengsten...,400.0,Johan Bakker (1889-1975),"Olieverf op doek, 50 x 60 cm, gesigneerd r.o. ...",https://images.kunstveiling.nl/tmp-6b47b8ba608...,"""url"": ""https://www.kunstveiling.nl/items/boer...",Bekijk item,https://www.kunstveiling.nl/items/johan-bakker...,50.0,60.0,Goed,Olieverf,Handgesigneerd,Ingelijst,62.0,72.0,1945 t/m 1999,Klassiek,Landschap,Johan Bakker - Boerderij te Warffum Groningen
4,1607956263-3145,https://www.kunstveiling.nl/veilingopbrengsten...,77,https://www.kunstveiling.nl/veilingopbrengsten...,140.0,Andre Gignac (1866-1956),Het werk is gesigneerd door Andre Gignac (1866...,https://images.kunstveiling.nl/tmp-7a70568b06f...,"""url"": ""https://www.kunstveiling.nl/items/ruig...",Bekijk item,https://www.kunstveiling.nl/items/andre-gignac...,48.0,68.0,Goed,Olieverf,Handgesigneerd,Ingelijst,64.0,84.0,1945 t/m 1999,Realistisch,Zeegezicht,Andre Gignac - Ruige zee
42,1607954736-2481,https://www.kunstveiling.nl/veilingopbrengsten...,32,https://www.kunstveiling.nl/veilingopbrengsten...,160.0,Frederic Benrath (1930-2007),zeer abstract werk in typische stijl,https://images.kunstveiling.nl/tmp-27e300230ea...,"""url"": ""https://www.kunstveiling.nl/items/abst...",Bekijk item,https://www.kunstveiling.nl/items/frederic-ben...,79.0,59.0,Zeer Goed,Olieverf,Handgesigneerd,Ingelijst,79.0,59.0,1945 t/m 1999,Abstract,Abstract,Frederic Benrath - abstracte kunst
43,1607955350-2749,https://www.kunstveiling.nl/veilingopbrengsten...,38,https://www.kunstveiling.nl/veilingopbrengsten...,125.0,Henk Poeder (1964),Prachtig olieverf gemaakt door h poeder betre...,https://images.kunstveiling.nl/tmp-083b3dd550a...,"""url"": ""https://www.kunstveiling.nl/items/ysvo...",Bekijk item,https://www.kunstveiling.nl/items/henk-poeder-...,28.0,20.0,Goed,Olieverf,Handgesigneerd,Ingelijst,37.0,30.0,2000 t/m Heden,Realistisch,Dieren,Henk Poeder - Ysvogel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8849,1607956318-3169,https://www.kunstveiling.nl/veilingopbrengsten...,87,https://www.kunstveiling.nl/veilingopbrengsten...,200.0,Paul Messely (1927),In perfecte staat\nRomantisch\nFijn afgewerkt,https://images.kunstveiling.nl/tmp-f4a61af9660...,"""url"": ""https://www.kunstveiling.nl/items/bloe...",Bekijk item,https://www.kunstveiling.nl/items/paul-messely...,80.0,50.0,Zeer Goed,Olieverf,Handgesigneerd,Ingelijst,90.0,60.0,1945 t/m 1999,Romantisch,Bloemen en Planten,Paul Messely - Bloemen stilleven
8852,1607958948-4308,https://www.kunstveiling.nl/veilingopbrengsten...,220,https://www.kunstveiling.nl/veilingopbrengsten...,1000.0,Hessel de Boer (1921-2003),De kunstenaar Hessel de\nBoer is een Haagse sc...,https://images.kunstveiling.nl/tmp-2578757da65...,"""url"": ""https://www.kunstveiling.nl/items/olie...",Bekijk item,https://www.kunstveiling.nl/items/hessel-de-bo...,66.0,77.0,Zeer Goed,Olieverf,Handgesigneerd,Ingelijst,80.0,91.0,1945 t/m 1999,Klassiek,Figuren,"Hessel de Boer - Olieverf op doek, Vrouw op ee..."
8854,1607969198-8773,https://www.kunstveiling.nl/veilingopbrengsten...,8,https://www.kunstveiling.nl/veilingopbren

In [22]:
data.to_csv("no_nan_kunstveiling_nl_txmm.csv", index=False)